In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square
from time import sleep
import math
import numpy as np
from time import sleep

mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))

tau = 0.005
tend = 0.1
pi = math.pi
u0 = sin(pi*x)*sin(pi*y)

fes = H1(mesh, order=3,dirichlet="bottom|right|left|top")
u,v = fes.TnT()
mform = u*v*dx
aform = grad(u)*grad(v)*dx

m = BilinearForm(mform).Assemble()
a = BilinearForm(aform).Assemble()
mstar = BilinearForm(mform+tau*aform).Assemble()
mstarinv = mstar.mat.Inverse(fes.FreeDofs())

gfu = GridFunction(fes)
gfu.Set(u0)

t = Parameter(0.0)
funcf = -sin(pi*x)*sin(pi*y)*cos(t) + 2*pi**2*sin(pi*x)*sin(pi*y)*cos(t)
#gff = GridFunction(fes)
gff = GridFunction(L2(mesh,order=gfu.space.globalorder+1))
gfft = GridFunction(gff.space,multidim=0)
time = 0.0

#scene = Draw(gff, interpolate_multidim=True, animate=True, autoscale=False, min=-10, max=20)
#sleep(3)

for i in range(7):
    t.Set(i/5)
    gff.Set(funcf)
    #scene.Redraw()
    #sleep(1)
    gfft.AddMultiDimComponent(gff.vec)
#Draw(gfft, mesh, interpolate_multidim=True, animate=True)

gffout = GridFunction(L2(mesh,order=gfu.space.globalorder+1))
#gffout.set()
scene = Draw(gffout, autoscale=False, min=-10, max=20)
sleep(3)

for j in range(7):
    gffout.Set(gfft.MDComponent(j))
    scene.Redraw()
    #Draw(gffout, mesh, interpolate_multidim=True, animate=True)
    sleep(0.5)
    
#Draw(gfft, mesh, interpolate_multidim=True, animate=True, autoscale=False, min=-1, max=1)

#Draw(gfu,mesh)
#print(fes.ndof)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [38]:
help(GridFunction)

Help on class GridFunction in module ngsolve.comp:

class GridFunction(ngsolve.fem.CoefficientFunction)
 |  a field approximated in some finite element space
 |   Keyword arguments can be:
 |  multidim: 
 |   Multidimensional GridFunction
 |  nested: bool = False
 |   Generates prolongation matrices for each mesh level and prolongates
 |   the solution onto the finer grid after a refinement.
 |  autoupdate: 
 |   Automatically update on FE space update
 |  
 |  Method resolution order:
 |      GridFunction
 |      ngsolve.fem.CoefficientFunction
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  AddMultiDimComponent(...)
 |      AddMultiDimComponent(self: ngsolve.comp.GridFunction, arg0: ngsolve.la.BaseVector) -> None
 |  
 |  CF(...)
 |      CF(self: ngsolve.comp.GridFunction, diffop: ngsolve.fem.DifferentialOperator) -> ngsolve.fem.CoefficientFunction
 |      
 |      
 |      
 |      Parameters:
 |      
 |      diffop : ngsolv

In [5]:
import numpy as np
a = np.zeros((3,2))
print(a)

[[0. 0.]
 [0. 0.]
 [0. 0.]]


In [1]:
for i in range(3):
    print(i)

0
1
2


In [66]:
Draw?

Signature: Draw(obj, *args, show=True, **kwargs)
Docstring: <no docstring>
File:      ~/.local/lib/python3.10/site-packages/netgen/webgui.py
Type:      function

In [100]:
# basic code for parabolic equation 
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square
from time import sleep
import math
import numpy as np
from time import sleep

mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))

tau = 0.005
tend = 0.1
pi = math.pi
u0 = sin(pi*x)*sin(pi*y)

fes = H1(mesh, order=3,dirichlet="bottom|right|left|top")
u,v = fes.TnT()
mform = u*v*dx
aform = grad(u)*grad(v)*dx

m = BilinearForm(mform).Assemble()
a = BilinearForm(aform).Assemble()
mstar = BilinearForm(mform+tau*aform).Assemble()
mstarinv = mstar.mat.Inverse(fes.FreeDofs())

gfu = GridFunction(fes)
gfu.Set(u0)

t = Parameter(0.0)
funcf = -sin(pi*x)*sin(pi*y)*sin(t) + 2*pi**2*sin(pi*x)*sin(pi*y)*cos(t)
f = LinearForm(fes)
f += funcf*v*dx

Nt = int(tend/tau)
gfut = GridFunction(gfu.space, multidim=0)
gfut.AddMultiDimComponent(gfu.vec)
for j in range(Nt):
    t.Set((j+1)*tau)
    #print(t.Get())
    f.Assemble()
    res = f.vec - a.mat * gfu.vec
    w = mstarinv * res
    gfu.vec.data += tau*w
    gfut.AddMultiDimComponent(gfu.vec)

uexa = sin(pi*x)*sin(pi*y)*cos(t)
errv = np.zeros(Nt)
for k in range(Nt):
    t.Set((k+1)*tau)
    errv[k] = math.sqrt(Integrate((gfut.MDComponent(k)-uexa)**2, mesh, order=5))

print(max(errv))

# time=0.0
# gfuout = GridFunction(fes)
# scene = Draw(gfuout, autoscale=False, min=-1, max=2)
# sleep(2)

# for k in range(Nt):
#     gfuout.Set(gfut.MDComponent(k))
#     scene.Redraw()
#     sleep(0.1)

0.0001907127549234598


In [107]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
help(mesh)

Help on Mesh in module ngsolve.comp object:

class Mesh(pybind11_builtins.pybind11_object)
 |  NGSolve interface to the Netgen mesh. Provides access and functionality
 |  to use the mesh for finite element calculations.
 |  
 |  Parameters:
 |  
 |  mesh (netgen.Mesh): a mesh generated from Netgen
 |  
 |  Method resolution order:
 |      Mesh
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  BBBoundaries(...)
 |      BBBoundaries(self: ngsolve.comp.Mesh, pattern: str) -> ngsolve.comp.Region
 |      
 |      Return co dim 3 boundary mesh-region matching the given regex pattern
 |  
 |  BBoundaries(...)
 |      BBoundaries(self: ngsolve.comp.Mesh, pattern: str) -> ngsolve.comp.Region
 |      
 |      Return co dim 2 boundary mesh-region matching the given regex pattern
 |  
 |  Boundaries(...)
 |      Boundaries(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. Boundaries(self: ngsolve.comp.Mesh, pattern: str) -> ng

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square
from time import sleep
import math
import numpy as np
from time import sleep

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

tau = 0.001
tend = 1
pi = math.pi
u0 = sin(pi*x)*sin(pi*y)

fes = H1(mesh, order=1,dirichlet="bottom|right|left|top")
u,v = fes.TnT()
mform = u*v*dx
aform = grad(u)*grad(v)*dx

m = BilinearForm(mform)
a = BilinearForm(aform)
mstar = BilinearForm(mform+tau*aform)

gfu = GridFunction(fes)
gfu.Set(u0)
gfut = GridFunction(gfu.space, multidim=0)
gfut.AddMultiDimComponent(gfu.vec)

t = Parameter(0.0)
uexa = sin(pi*x)*sin(pi*y)*cos(t)
funcf = -sin(pi*x)*sin(pi*y)*sin(t) + 2*pi**2*sin(pi*x)*sin(pi*y)*cos(t)
f = LinearForm(fes)
f += funcf*v*dx

level = 0
Nt = int(tend/tau)

def TimeStepping():
    for j in range(Nt):
        t.Set((j+1)*tau)
        f.Assemble()
        res = f.vec - a.mat * gfu.vec
        w = mstarinv * res
        gfu.vec.data += tau*w
        gfut.AddMultiDimComponent(gfu.vec)

l = []
while level < 6:
    m.Assemble()
    a.Assemble()
    mstar.Assemble()
    mstarinv = mstar.mat.Inverse(fes.FreeDofs())
    
    TimeStepping()
    err = math.sqrt(Integrate((gfut.MDComponent(-1)-uexa)**2, mesh, order=5))
    print("level = ", level, "err = ", err)
    l.append((level,err))
    
    mesh.Refine()
    level += 1
    
    fes.Update()
    gfu.Update()
    gfu.Set(u0)
    gfut = GridFunction(gfu.space, multidim=0)
    gfut.AddMultiDimComponent(gfu.vec) 




# def errestimator()
#     errv = np.zeros(Nt)
#     for k in range(Nt):
#         t.Set((k+1)*tau)
#         errv[k] = math.sqrt(Integrate((gfut.MDComponent(k)-uexa)**2, mesh, order=5))
#     return max(errv)